In [ ]:
# 1. Import libraries
from google.cloud import bigquery
import vertexai
# We import the BigQuery client to manage data and Vertex AI to manage models.

# 2. Configuration Variables
PROJECT_ID = "qwiklabs-gcp-03-5dc51bd29ec6"
REGION = "US"
DATASET_ID = "aurora_bay_dataset"
CONN_NAME = "embedding_conn"
# These variables ensure we target the specific project, location, and connection created for this lab.

# 3. Initialize Clients
client = bigquery.Client(project=PROJECT_ID, location=REGION)
# Establishes the authenticated connection to BigQuery so we can run SQL commands.

vertexai.init(project=PROJECT_ID, location="us-central1")
# Initializes the Vertex AI SDK (often required for the underlying API calls).

# 4. Create Dataset
dataset_ref = client.create_dataset(DATASET_ID, exists_ok=True)
# Creates the folder (Dataset) in BigQuery to hold our tables; 'exists_ok' prevents errors if it's already there.

print(f"Configuration updated. Region: {REGION}, Connection: {CONN_NAME}")

Configuration updated. Region: US, Connection: embedding_conn


In [ ]:
# 5. Define Source URI and Table Reference
uri = "gs://labs.roitraining.com/aurora-bay-faqs/aurora-bay-faqs.csv"
table_ref = dataset_ref.table(TABLE_NAME)
# Points to the source CSV file in the Google Cloud Storage bucket and defines the destination table.

# 6. Configure the Load Job
job_config = bigquery.LoadJobConfig(
    source_format=bigquery.SourceFormat.CSV,
    skip_leading_rows=1,
    autodetect=True,
    write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE
)
# Configures the job to handle CSVs, skip the header row, auto-guess column types, and overwrite existing data.

# 7. Run the Load Job
load_job = client.load_table_from_uri(uri, table_ref, job_config=job_config)
# Triggers the actual transfer of data from Storage to BigQuery.

load_job.result()
# Forces the code to pause and wait until the data load is 100% complete before moving on.

destination_table = client.get_table(table_ref)
print(f"Loaded {destination_table.num_rows} rows into {DATASET_ID}.{TABLE_NAME}.")
# Verifies the load by printing the row count.

Starting load job from gs://labs.roitraining.com/aurora-bay-faqs/aurora-bay-faqs.csv...
Loaded 50 rows into aurora_bay_dataset.faqs.


In [ ]:
# Query the first 45 rows of the table we just loaded
validation_query = f"""
SELECT * FROM `{PROJECT_ID}.{DATASET_ID}.{TABLE_NAME}`
LIMIT 45
"""
# Defines a SQL query to fetch exactly 45 rows so we can verify the data volume and content.

print(f"Inspecting table: {PROJECT_ID}.{DATASET_ID}.{TABLE_NAME}")
# Prints the target table ID to the console for tracking purposes.

df = client.query(validation_query).to_dataframe()
# 1. Sends the query to BigQuery.
# 2. Downloads the results.
# 3. Loads them into a Pandas DataFrame ('df') for display.

# Display the data
print("\n--- Column Names ---")
print(df.columns.tolist())
# Prints the column headers (e.g., 'string_field_0') to identify if we need to rename them later.

print("\n--- First 45 Rows ---")
print(df.to_markdown(index=False))
# Formats and prints the entire 45-row dataframe as a clean Markdown table.

Inspecting table: qwiklabs-gcp-03-5dc51bd29ec6.aurora_bay_dataset.faqs

--- Column Names ---
['string_field_0', 'string_field_1']

--- First 45 Rows ---
| string_field_0                                                 | string_field_1                                                                                                                                                                               |
|:---------------------------------------------------------------|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| When was Aurora Bay founded?                                   | Aurora Bay was founded in 1901 by a group of fur traders who recognized the region’s strategic coastal location.                                                                             |
| What is the population of Aurora Bay?                          | Aurora

In [ ]:
# --- Configuration ---
# Ensure these match your current setup
REGION = "US"
CONN_NAME = "embedding_conn"
# Sets the region to the US multi-region and defines the connection ID to talk to Vertex AI.

# 9. Create the Remote Model "Kyber"
# (If this already ran successfully, re-running it is fine/harmless)
create_model_sql = f"""
CREATE OR REPLACE MODEL `{PROJECT_ID}.{DATASET_ID}.{EMBEDDING_MODEL_NAME}`
REMOTE WITH CONNECTION `{PROJECT_ID}.{REGION}.{CONN_NAME}`
OPTIONS (endpoint = 'text-embedding-004');
"""
# SQL command to create a "Remote Model" in BigQuery that acts as a gateway to the Vertex AI 'text-embedding-004' API.

print(f"Creating remote model '{EMBEDDING_MODEL_NAME}'...")
model_job = client.query(create_model_sql)
# Sends the CREATE MODEL command to BigQuery.

model_job.result()
# Waits for the model creation to finish before proceeding.

print(f"Model '{EMBEDDING_MODEL_NAME}' ready.")

# 10. Generate Embeddings (FIXED)
# We map 'string_field_0' -> 'question' and 'string_field_1' -> 'answer'
create_embeddings_sql = f"""
CREATE OR REPLACE TABLE `{PROJECT_ID}.{DATASET_ID}.{TABLE_NAME}_embeddings` AS
SELECT
  *
FROM ML.GENERATE_EMBEDDING(
  MODEL `{PROJECT_ID}.{DATASET_ID}.{EMBEDDING_MODEL_NAME}`,
  (
    SELECT
      string_field_0 AS question,
      string_field_1 AS answer,
      CONCAT('Question: ', string_field_0, ' Answer: ', string_field_1) as content
    FROM `{PROJECT_ID}.{DATASET_ID}.{TABLE_NAME}`
  )
);
"""
# 1. Selects raw data and renames generic columns (string_field_0) to meaningful names (question).
# 2. Concatenates text into a single 'content' column for the AI to read.
# 3. Calls the embedding model to convert that text into vectors.
# 4. Saves everything into a new table ending in '_embeddings'.

print("Generating embeddings with corrected column names...")
embedding_job = client.query(create_embeddings_sql)
# Triggers the batch job to generate embeddings for all rows.

embedding_job.result()
# Waits for the embedding generation to complete (this usually takes about 30-60 seconds).

print(f"✅ Embeddings generated and stored in '{TABLE_NAME}_embeddings'.")

Creating remote model 'Kyber'...
Model 'Kyber' ready.
Generating embeddings with corrected column names...
✅ Embeddings generated and stored in 'faqs_embeddings'.


In [ ]:
import json

# --- Configuration ---
REGION = "US"
CONN_NAME = "embedding_conn"
GEN_MODEL_NAME = "gemini_flash_lab"

def rag_chatbot(user_query):
    print(f"\n" + "="*60)
    print(f"👤 User asks: {user_query}")
    print("-" * 60)

    # A. RETRIEVAL (Vector Search)
    search_sql = f"""
    SELECT base.question, base.answer, distance
    FROM VECTOR_SEARCH(
      TABLE `{PROJECT_ID}.{DATASET_ID}.{TABLE_NAME}_embeddings`,
      'ml_generate_embedding_result',
      (
        SELECT ml_generate_embedding_result AS embedding
        FROM ML.GENERATE_EMBEDDING(
          MODEL `{PROJECT_ID}.{DATASET_ID}.{EMBEDDING_MODEL_NAME}`,
          (SELECT '{user_query}' AS content)
        )
      ),
      top_k => 3
    )
    """
    search_results = client.query(search_sql).to_dataframe()

    # B. CONTEXT CONSTRUCTION
    context_text = ""
    for index, row in search_results.iterrows():
        context_text += f"Match {index+1}: Q: {row['question']} | A: {row['answer']}\n"

    # C. GENERATION (Dynamic Suggestion Prompt)
    prompt = f"""
    You are a helpful assistant for Aurora Bay.
    Use the Context provided below to answer the User Question.

    INSTRUCTIONS:
    1. If the Context contains the direct answer, provide it clearly.

    2. If the Context does NOT contain the direct answer, strictly follow this structure:

       - Start with: "I do not have any information to that specific question but here is what similar information that may be relevant to your question:"
       - Then, summarize the similar information found in the Context.

       - Header: "Suggestion:"
       - Content: Generate a logical next step for the user based on the specific entity they asked about.
         (Example: If they asked about the Library, suggest contacting the Library. If they asked about Permits, suggest checking the Town Hall website).

       - Closing: "Is there anything else I can help with?"

    Context:
    {context_text}

    User Question: {user_query}
    """

    # Robust SQL (Select *)
    generate_sql = f"""
    SELECT *
    FROM ML.GENERATE_TEXT(
      MODEL `{PROJECT_ID}.{DATASET_ID}.{GEN_MODEL_NAME}`,
      (SELECT @prompt AS prompt),
      STRUCT(
        0.5 AS temperature,
        1024 AS max_output_tokens,
        TRUE AS flatten_json_output
      )
    )
    """

    job_config = bigquery.QueryJobConfig(
        query_parameters=[bigquery.ScalarQueryParameter("prompt", "STRING", prompt)]
    )

    response_job = client.query(generate_sql, job_config=job_config)
    result = response_job.to_dataframe()

    # D. DYNAMIC COLUMN FINDER
    if 'ml_generate_text_result' in result.columns:
        raw_response = result.iloc[0]['ml_generate_text_result']
    elif 'ml_generate_text_llm_result' in result.columns:
        raw_response = result.iloc[0]['ml_generate_text_llm_result']
    else:
        raw_response = "Error: Could not find output column."

    # E. JSON PARSING
    final_answer = raw_response
    try:
        if str(raw_response).strip().startswith('{'):
            parsed = json.loads(raw_response)
            if 'candidates' in parsed and len(parsed['candidates']) > 0:
                parts = parsed['candidates'][0]['content']['parts']
                final_answer = parts[0]['text']
    except Exception as e:
        pass

    print(f"🤖 Chatbot:\n{final_answer}")
    return final_answer

# --- TEST SUITE: 5 SCENARIOS ---

# LIST 1: MATCHING QUESTIONS (Expect Direct Answers)
print("\n>>> TESTING MATCHING QUESTIONS (Expect Direct Answers)...")
_ = rag_chatbot("How can I volunteer for town events?")
_ = rag_chatbot("Where is the community center located?")
_ = rag_chatbot("When are the town council meetings held?")

# LIST 2: NON-MATCHING QUESTIONS (Expect Suggestions)
print("\n>>> TESTING MISSING DATA (Expect Suggestions)...")
_ = rag_chatbot("What time does the library host its book club for kids?")

# The underscore (_) captures the return value so Jupyter doesn't print it again
_ = rag_chatbot("How do I apply for a permit to build a fence?")


>>> TESTING MATCHING QUESTIONS (Expect Direct Answers)...

👤 User asks: How can I volunteer for town events?
------------------------------------------------------------
🤖 Chatbot:
You can sign up through the Community Center’s volunteer portal or attend volunteer meetings announced on the town’s website and local bulletin boards.

👤 User asks: Where is the community center located?
------------------------------------------------------------
🤖 Chatbot:
The Aurora Bay Community Center is located on Seaview Road.

👤 User asks: When are the town council meetings held?
------------------------------------------------------------
🤖 Chatbot:
Town council meetings are held every second Tuesday of the month at 6 PM in the Town Hall conference room. Meetings are open to the public.

>>> TESTING MISSING DATA (Expect Suggestions)...

👤 User asks: What time does the library host its book club for kids?
------------------------------------------------------------
🤖 Chatbot:
I do not have any info